In [4]:
import cv2
import numpy as np

# 'path to input image/video'

# 'path to yolo config file' 
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.cfg
CONFIG='./yolo/yolov3.cfg'

# 'path to text file containing class names'
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.txt
CLASSES='./yolo/yolov3.txt'

# 'path to yolo pre-trained weights' 
# wget https://pjreddie.com/media/files/yolov3.weights
WEIGHTS='./yolo/yolov3.weights'

In [5]:
import os  
print(os.path.exists(CLASSES))
print(os.path.exists(CONFIG))
print(os.path.exists(WEIGHTS))

True
True
True


In [6]:
classes = None
with open(CLASSES, 'r') as f:
     classes = [line.strip() for line in f.readlines()]
        
scale = 0.00392
conf_threshold = 0.5
nms_threshold = 0.4

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [7]:
def get_output_layers(net): 
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [8]:
def processImage(image, index):

    Width = image.shape[1]
    Height = image.shape[0]

    # read pre-trained model and config file
    net = cv2.dnn.readNet(WEIGHTS, CONFIG)

    # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                    
    # apply non-max suppression
    
    classified_objects = []
    for i in class_ids:
        classified_objects.append(classes[i])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
    
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    # display output image    
    
    #cv2.imshow(out_image_name, image)
    # wait until any key is pressed
    #cv2.waitKey()
    # save output image to disk
    return classified_objects

In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = 'alexandradaddario'
pr_classes = {}
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        print(directory + '/' + filename)
        temp = mpimg.imread(directory + '/' + filename)
        pr_classes[filename] = processImage(temp, filename)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


alexandradaddario/11176210_859954387410261_1985799031_n.jpg
alexandradaddario/23421155_325907524543476_3735144920583766016_n.jpg
alexandradaddario/11378391_944585778918991_1917638315_n.jpg
alexandradaddario/47586205_2261195664123134_5969231701740536350_n.jpg
alexandradaddario/11374002_1615738015368989_1991728466_n.jpg
alexandradaddario/13298032_492428684281391_1587643377_n.jpg
alexandradaddario/26868100_340709193081439_9106740417520467968_n.jpg
alexandradaddario/11850367_1027305420622130_560170916_n.jpg
alexandradaddario/41863396_1886803911416727_8213046552232438563_n.jpg
alexandradaddario/10362077_224306174445711_177294697_n.jpg
alexandradaddario/25025389_183599632385515_8686203436376522752_n.jpg
alexandradaddario/53006843_2419515061610153_57865838088577126_n.jpg
alexandradaddario/39314906_449174282155812_4994793480573157376_n.jpg
alexandradaddario/28752719_175468599742313_5048040359100153856_n.jpg
alexandradaddario/11358938_1432593413715328_496433075_n.jpg
alexandradaddario/10995258_

alexandradaddario/11352055_556646087810082_1864688025_n.jpg
alexandradaddario/23823471_496587524031259_408809869294960640_n.jpg
alexandradaddario/11246935_1588799984733690_332583555_n.jpg
alexandradaddario/23824087_316294855445244_8537593741119062016_n.jpg
alexandradaddario/15251822_1872018756353111_6406653147502084096_n.jpg
alexandradaddario/13256710_666997450104674_1694834509_n.jpg
alexandradaddario/17881354_1474636219224872_4977668075869437952_n.jpg
alexandradaddario/18722191_450160262010429_2169369113418268672_n.jpg
alexandradaddario/55818892_130257804753869_6031043946007064323_n.jpg
alexandradaddario/11419051_106814536323034_203110628_n.jpg
alexandradaddario/14727571_1092935514159669_7507489162803544064_n.jpg
alexandradaddario/1208254_225638140935475_1242243218_n.jpg
alexandradaddario/39952597_1731432580259391_3739143668280328192_n.jpg
alexandradaddario/44439655_1932053610242046_6845658239453953748_n.jpg
alexandradaddario/11333437_389824967888156_196636771_n.jpg
alexandradaddario/

alexandradaddario/49913275_982412138619307_4556591518574546489_n.jpg
alexandradaddario/22280183_466353370411402_2420350238939152384_n.jpg
alexandradaddario/11358229_1668262473393238_1560591376_n.jpg
alexandradaddario/11378352_668033043331865_206634255_n.jpg
alexandradaddario/1516338_1473936919502474_13697592_n.jpg
alexandradaddario/11201681_828877820528246_328998534_n.jpg
alexandradaddario/12139710_420516184823856_991160894_n.jpg
alexandradaddario/15802675_1827990744089221_4190734641658855424_n.jpg
alexandradaddario/25017291_1888950877799739_8996749990151847936_n.jpg
alexandradaddario/11410457_666363890130477_1458644531_n.jpg
alexandradaddario/11348085_516111085202924_1118667878_n.jpg
alexandradaddario/11356900_755102314635267_2002759718_n.jpg
alexandradaddario/37996317_1878113342270454_2584985556689616896_n.jpg
alexandradaddario/12354044_1644503302478968_693065964_n.jpg
alexandradaddario/11379015_1449936988653849_1023060108_n.jpg
alexandradaddario/22582184_1999076813681158_28115905566

alexandradaddario/927034_626994224045016_1052478363_n.jpg
alexandradaddario/32233027_1692213610861532_3582018568998354944_n.jpg
alexandradaddario/21434069_346415569104465_4302256916358234112_n.jpg
alexandradaddario/11324904_981898831850350_1260701691_n.jpg
alexandradaddario/927729_440824046082178_508387110_n.jpg
alexandradaddario/18722956_458272047839777_1500646012555886592_n.jpg
alexandradaddario/10369525_327996734016157_417774593_n.jpg
alexandradaddario/44509962_1123323801164867_2428720588652147822_n.jpg
alexandradaddario/10401588_566247560162780_404729186_n.jpg
alexandradaddario/23279995_1754151448227388_2146941361134239744_n.jpg
alexandradaddario/30084948_2032251700365604_1549787787111170048_n.jpg
alexandradaddario/49907235_2156834611295364_4127965694888855108_n.jpg
alexandradaddario/11085087_729462940504781_747531096_n.jpg
alexandradaddario/20184473_111915896126042_8815145814960635904_n.jpg
alexandradaddario/12237224_857085397741910_738858749_n.jpg
alexandradaddario/10747889_35826

...

alexandradaddario/1171246_754178614682092_779325636_n.jpg
alexandradaddario/45457177_256554708546261_13453640976765093_n.jpg
alexandradaddario/51465054_118528115934616_8641183108634080283_n.jpg
alexandradaddario/51236359_295636071124704_1868030521428171928_n.jpg
alexandradaddario/36483805_532717543813940_8954169791753486336_n.jpg
alexandradaddario/18645549_274280549702240_1569146789557174272_n.jpg
alexandradaddario/928880_947811315263858_1922582610_n.jpg
alexandradaddario/1391242_218643824963885_456995516_n.jpg
alexandradaddario/15034919_1138459426237420_8267667467610882048_n.jpg
alexandradaddario/11372281_1445137735804277_994977535_n.jpg
alexandradaddario/37035167_248692812522379_5792342965837889536_n.jpg
alexandradaddario/10175290_562023243895222_1983218780_n.jpg
alexandradaddario/10507803_1447795035479633_278504867_n.jpg
alexandradaddario/11325822_1671526746411518_2007359296_n.jpg
alexandradaddario/1208366_177301129130455_2046267348_n.jpg
alexandradaddario/14156188_176207899479610_2

alexandradaddario/10693634_616334021809005_1199255786_n.jpg
alexandradaddario/10467747_1431146673827369_311399153_n.jpg
alexandradaddario/18581200_212570565916753_6954962433584660480_n.jpg
alexandradaddario/21985455_735846199935637_9116508715044831232_n.jpg
alexandradaddario/10513791_502808326487613_589814205_n.jpg
alexandradaddario/50295439_316412125747532_6815331087520488865_n.jpg
alexandradaddario/27575447_788377098017809_4499841127259570176_n.jpg
alexandradaddario/1169170_508466265917885_890273383_n.jpg
alexandradaddario/11311667_1429543964035429_543069168_n.jpg
alexandradaddario/15338340_1493460680671819_4830727735615684608_n.jpg
alexandradaddario/17127021_193230124500572_4244769088338919424_n.jpg
alexandradaddario/11256779_641598349318093_655778492_n.jpg
alexandradaddario/29401393_843708559170943_5479434609546493952_n.jpg
alexandradaddario/11380939_403012209904291_285778222_n.jpg
alexandradaddario/41248853_481095192370349_3510420259822332030_n.jpg
alexandradaddario/1739658_252107

alexandradaddario/15043973_192740954516777_3728806949014208512_n.jpg
alexandradaddario/916473_846384438712069_33126488_n.jpg
alexandradaddario/11259911_374562052742397_1248121563_n.jpg
alexandradaddario/10748147_815979355109780_1232658192_n.jpg
alexandradaddario/11242628_1452779328355878_297549202_n.jpg
alexandradaddario/18809178_231355897356662_5626871667062996992_n.jpg
alexandradaddario/14712320_355256884819435_2984460606624497664_n.jpg
alexandradaddario/11850211_424184127776379_2112456212_n.jpg
alexandradaddario/36914253_435505860260641_4309987337799991296_n.jpg
alexandradaddario/14448313_267611463633126_6599408247499653120_n.jpg
alexandradaddario/44491300_144257313218101_675589621755442920_n.jpg
alexandradaddario/1922307_661219757290931_1124362965_n.jpg
alexandradaddario/18382180_1327533370699737_6798298055444529152_n.jpg
alexandradaddario/11358216_1468714900105643_1975399044_n.jpg
alexandradaddario/11421803_413182632196725_650953725_n.jpg
alexandradaddario/10544050_698234470241601

alexandradaddario/24125598_502875540089821_8483274215984726016_n.jpg
alexandradaddario/12070890_637548573053554_1261844768_n.jpg
alexandradaddario/11325139_936106263076696_1656125912_n.jpg
alexandradaddario/11426193_1764028253824816_386483630_n.jpg
alexandradaddario/12063085_1552402755089911_1608409453_n.jpg
alexandradaddario/29414537_368110577023910_1364733473281015808_n.jpg
alexandradaddario/17076380_1261697360573279_6548051502475247616_n.jpg
alexandradaddario/36892755_227876821156374_5454843258897170432_n.jpg
alexandradaddario/18444507_1904261983187330_3508425335173545984_n.jpg
alexandradaddario/927274_624600717617553_1879361211_n.jpg
alexandradaddario/33041153_425225907942988_5550260845050593280_n.jpg
alexandradaddario/11335622_1593068667597974_151768550_n.jpg
alexandradaddario/1515786_757913020900680_587820343_n.jpg
alexandradaddario/11856736_1628536787420784_1561257346_n.jpg
alexandradaddario/14128738_1289860814359753_1852785601_n.jpg
alexandradaddario/11005129_1558695284380921_1

alexandradaddario/11244432_839831509435870_1158623794_n.jpg
alexandradaddario/1538493_693777894028329_1348552558_n.jpg
alexandradaddario/13151363_1594302634214475_837482046_n.jpg
alexandradaddario/12750330_1694239550848899_1313561339_n.jpg
alexandradaddario/16123057_367740046926790_1699880684899794944_n.jpg
alexandradaddario/11334633_846310468783174_1616363392_n.jpg
alexandradaddario/917124_1436331153289634_1529578324_n.jpg


In [10]:
import csv
w = csv.writer(open("AOC_output.csv", "w"))
for key, val in pr_classes.items():
    w.writerow([key, val])